# Amadey Loader
> Analysis of CPP Amadey loader

- toc: true 
- badges: true
- categories: [cpp,stl,amadey,loader,config]


## Overview

### Sample

SHA256 `18A38FA6F5B306243D99621556AF948A61DAED29619AB755E25010F9E254C6BD`

PDB (lol) `D:\Mktmp\Amadey\Release\Amadey.pdb`

### References
- [LockBit 3.0 Being Distributed via Amadey Bot](https://asec.ahnlab.com/en/41450/)
- [SmokeLoader Malware Used to Augment Amadey Infostealer](https://blogs.blackberry.com/en/2022/07/smokeloader-malware-used-to-augment-amadey-infostealer)
- [Analyzing Amadey – a simple native malware](https://krabsonsecurity.com/2019/02/13/analyzing-amadey-a-simple-native-malware/)

### TODO / Future Work
Since there are usually going to be global strings in CPP maybe it would be worth writing a small script to label all the global string addresses?
- iterate _PVFV (what I call the constructor Vtable lololol so wrong)
- for each fn if there is a string assign
- assume the args are int he same position (or maybe try hexrays helper fns)
- grab the char* string and label the global str::string object and set the type


## Analysis

### String Decryption
- strings look like base64
- string transform



In [8]:
import base64

str_hash_data = 'd6052c4fe86a6346964a6bbbe2423e20'
str_data = 'QOqdDVR RQLzIo=='
str_data = 'XWaNMGOS1BI18fCg iKD7361gd2jM6MP0zCp3pKDfZscLU2oaCO0UYKP2NUcPIEkeSqV5IODXjAtUI=='
str_alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '

str_hash = ''

for i in range(len(str_data)):
    str_hash += str_hash_data[i % len(str_hash_data)]

out = ''

for i in range(len(str_data)):
    if str_data[i] not in str_alphabet:
        out += str_data[i]
        continue
    alphabet_count = str_alphabet.find(str_data[i])
    hash_count = str_alphabet.find(str_hash[i])
    index_calc = (alphabet_count + len(str_alphabet) - hash_count) % len(str_alphabet)
    out += str_alphabet[index_calc]

base64.b64decode(out)

b'SYSTEM\\CurrentControlSet\\Control\\ComputerName\\ComputerName'

In [10]:
def decrypt(str_data, str_hash_data, str_alphabet):
    str_hash = ''

    for i in range(len(str_data)):
        str_hash += str_hash_data[i % len(str_hash_data)]

    out = ''

    for i in range(len(str_data)):
        if str_data[i] not in str_alphabet:
            out += str_data[i]
            continue
        alphabet_count = str_alphabet.find(str_data[i])
        hash_count = str_alphabet.find(str_hash[i])
        index_calc = (alphabet_count + len(str_alphabet) - hash_count) % len(str_alphabet)
        out += str_alphabet[index_calc]

    return base64.b64decode(out)

decrypt('1RydQIOr3Zcp6emn RYv8IGzgUKS6r5ThSdqDVBERAP2Ir 0JQ1=', str_hash_data, str_alphabet)

b'abcdefghijklmnopqrstuvwxyz0123456789-_'

In [22]:
import re
str_hash_data = 'd6052c4fe86a6346964a6bbbe2423e20'
str_alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 '


def is_ascii(s):
    return all(c < 128 or c == 0 for c in s)

file_data = open('/tmp/amadey.bin','rb').read()

strings = []
for m in re.finditer(rb'[a-zA-Z =0-9]{4,}',file_data):
    strings.append(m.group().decode('utf-8'))

for s in strings:
    try:
        temp = decrypt(s, str_hash_data, str_alphabet)
        if is_ascii(temp) and len(temp) > 3:
            print(temp.decode('utf-8'))
    except:
        continue

193.56.146.243
/h8V2cQlbd3/index.php
3.50
50c1695437
rovwer.exe
SCHTASKS
/Create /SC MINUTE /MO 1 /TN 
 /TR "
" /F
SOFTWARE\Microsoft\Windows\CurrentVersion\RunOnce
SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\User Shell Folders
Startup
cmd /C RMDIR /s/q 
SOFTWARE\Microsoft\Windows\CurrentVersion\Run
rundll32 
 /Delete /TN "
Programs
SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders
%USERPROFILE%
\App
POST
&vs=
&sd=
&os=
&bi=
&ar=
&pc=
&un=
&dm=
&av=
&lv=
&og=
cred.dll|
Main
http://
https://
Plugins/
&unit=
shell32.dll
kernel32.dll
GetNativeSystemInfo
ProgramData\
AVAST Software
Avira
Kaspersky Lab
ESET
Panda Security
Doctor Web
360TotalSecurity
Bitdefender
Norton
Sophos
Comodo
WinDefender
0123456789
Content-Type: multipart/form-data; boundary=----
------

Content-Disposition: form-data; name="data"; filename="
"
Content-Type: application/octet-stream



------
--

?scr=1
.jpg
Content-Type: application/x-www-form-urlencoded
SYSTEM\CurrentControlSet\Control\Computer

In [23]:
hashes= ['d6052c4fe86a6346964a6bbbe2423e20',
        'bf045808586a2473c5a7441da6f3bfa9',
        'bc4cb0',
        'b6fd4be15536ff986cddb445cb644cf8']

